In [1]:
import os
import random
from PIL import Image
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Set random seed for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Load Donut model and processor
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base").to('cuda')

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, processor, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = np.array(image)
            image = self.transform(image=image)['image']
            # Convert image back to [0, 1] range
            image = (image - image.min()) / (image.max() - image.min())

        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        return pixel_values.squeeze(0), target

# Train one epoch
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for pixel_values, targets in pbar:
        pixel_values = pixel_values.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        outputs = model(pixel_values=pixel_values, decoder_input_ids=targets)
        logits = outputs.logits
        loss = loss_fn(logits.view(-1, model.config.vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(logits.argmax(dim=-1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data config
data_path = 'dj/data/'

# Training config
img_size = 480  # Default size used in Donut
LR = 1e-4
EPOCHS = 5
BATCH_SIZE = 4
num_workers = 4

# Augmentation for training transform
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Augmentation for testing transform
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Dataset definition
trn_dataset = ImageDataset(
    "dj/data/train.csv",
    "dj/data/train/",
    processor=processor,
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "dj/data/sample_submission.csv",
    "dj/data/test/",
    processor=processor,
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

# DataLoader definition
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

# Load model
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base").to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

# Training loop
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
        log += f"{k}: {v:.4f}\n"
    print(log)

# Evaluation
model.eval()
preds_list = []

for pixel_values, _ in tqdm(tst_loader):
    pixel_values = pixel_values.to(device)

    with torch.no_grad():
        outputs = model(pixel_values=pixel_values)
    preds_list.extend(outputs.logits.argmax(dim=-1).detach().cpu().numpy())

# Save predictions
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

sample_submission_df = pd.read_csv("dj/data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

pred_df.to_csv("pred3.csv", index=False)
pred_df.head()


1570 3140


  0%|          | 0/393 [00:00<?, ?it/s]It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
  0%|          | 0/393 [00:03<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 600.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 388.81 MiB is free. Process 58139 has 23.30 GiB memory in use. Of the allocated memory 22.45 GiB is allocated by PyTorch, and 560.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

1570 3140


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 16.81 MiB is free. Process 60312 has 23.66 GiB memory in use. Of the allocated memory 23.34 GiB is allocated by PyTorch, and 22.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [2]:
import os
import random
from PIL import Image
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from transformers import DonutProcessor, VisionEncoderDecoderModel
from torch.cuda.amp import autocast, GradScaler

# Set random seed for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Clear CUDA cache before loading the model
torch.cuda.empty_cache()

# Load Donut model and processor
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

# Move model to GPU with half precision if supported
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device, dtype=torch.float16 if torch.cuda.is_available() else torch.float32)

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, processor, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = np.array(image)
            image = self.transform(image=image)['image']
            # Convert image back to [0, 1] range
            image = (image - image.min()) / (image.max() - image.min())

        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        return pixel_values.squeeze(0), target

# Train one epoch with gradient accumulation and mixed precision
def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler, accumulation_steps):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    optimizer.zero_grad()
    pbar = tqdm(loader)
    for i, (pixel_values, targets) in enumerate(pbar):
        pixel_values = pixel_values.to(device, dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
        targets = targets.to(device)

        with autocast():
            outputs = model(pixel_values=pixel_values, decoder_input_ids=targets)
            logits = outputs.logits
            loss = loss_fn(logits.view(-1, model.config.vocab_size), targets.view(-1)) / accumulation_steps
        
        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        train_loss += loss.item() * accumulation_steps
        preds_list.extend(logits.argmax(dim=-1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item() * accumulation_steps:.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# Data config
data_path = 'dj/data/'

# Training config
img_size = 480  # Default size used in Donut
LR = 1e-4
EPOCHS = 5
BATCH_SIZE = 1  # Reduced batch size to 1
num_workers = 4
accumulation_steps = 4  # Gradient accumulation steps

# Augmentation for training transform
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Augmentation for testing transform
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Dataset definition
trn_dataset = ImageDataset(
    "dj/data/train.csv",
    "dj/data/train/",
    processor=processor,
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "dj/data/sample_submission.csv",
    "dj/data/test/",
    processor=processor,
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

# DataLoader definition
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)
scaler = GradScaler()

# Training loop
for epoch in range(EPOCHS):
    torch.cuda.empty_cache()
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device, scaler=scaler, accumulation_steps=accumulation_steps)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
        log += f"{k}: {v:.4f}\n"
    print(log)

# Evaluation
model.eval()
preds_list = []

for pixel_values, _ in tqdm(tst_loader):
    pixel_values = pixel_values.to(device, dtype=torch.float16 if torch.cuda.is_available() else torch.float32)

    with torch.no_grad():
        with autocast():
            outputs = model(pixel_values=pixel_values)
    preds_list.extend(outputs.logits.argmax(dim=-1).detach().cpu().numpy())

# Save predictions
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

sample_submission_df = pd.read_csv("dj/data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

pred_df.to_csv("pred3.csv", index=False)
pred_df.head()


/tmp/ipykernel_30093/1085792781.py:165: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


1570 3140


  0%|          | 0/1570 [00:00<?, ?it/s]It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
/tmp/ipykernel_30093/1085792781.py:76: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
  0%|          | 0/1570 [00:01<?, ?it/s]


ValueError: not enough values to unpack (expected 2, got 1)

In [3]:
import torch
torch.cuda.empty_cache()

In [2]:
import os
import random
from PIL import Image
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Set random seed for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Load Donut processor and model
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.eval()

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = np.array(image)
            image = self.transform(image=image)['image']
            # Convert image back to [0, 1] range
            image = (image - image.min()) / (image.max() - image.min())

        return image, torch.tensor(target, dtype=torch.long)

# Train one epoch
def train_one_epoch(loader, model, optimizer, loss_fn, device, processor):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for images, targets in pbar:
        images = images.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        
        # Prepare inputs for the model
        pixel_values = images
        task_prompt = "<s_rvlcdip>"
        decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

        outputs = model(pixel_values=pixel_values, decoder_input_ids=decoder_input_ids.to(device), labels=targets)
        loss = outputs.loss
        logits = outputs.logits
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(logits.argmax(dim=-1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data config
data_path = 'dj/data/'

# Training config
img_size = 480  # Default size used in Donut
LR = 1e-4
EPOCHS = 5
BATCH_SIZE = 8  # Reduced batch size to 1
num_workers = 4

# Augmentation for training transform
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Augmentation for testing transform
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Dataset definition
trn_dataset = ImageDataset(
    "dj/data/train.csv",
    "dj/data/train/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "dj/data/sample_submission.csv",
    "dj/data/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

# DataLoader definition
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

# Training loop
for epoch in range(EPOCHS):
    torch.cuda.empty_cache()
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device, processor=processor)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
        log += f"{k}: {v:.4f}\n"
    print(log)

# Evaluation
model.eval()
preds_list = []

for images, _ in tqdm(tst_loader):
    images = images.to(device)

    with torch.no_grad():
        outputs = model(pixel_values=images)
    preds_list.extend(outputs.logits.argmax(dim=-1).detach().cpu().numpy())

# Save predictions
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

sample_submission_df = pd.read_csv("dj/data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

pred_df.to_csv("pred_3.csv", index=False)
pred_df.head()


/opt/conda/envs/cv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/cv/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via

1570 3140


  0%|          | 0/197 [00:01<?, ?it/s]


ValueError: Expected input batch_size (1) to match target batch_size (8).

In [4]:
import os
import random
from PIL import Image
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Set random seed for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Load Donut processor and model
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.eval()

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = np.array(image)
            image = self.transform(image=image)['image']
            # Convert image back to [0, 1] range
            image = (image - image.min()) / (image.max() - image.min())

        return image, torch.tensor(target, dtype=torch.long)

# Train one epoch
def train_one_epoch(loader, model, optimizer, loss_fn, device, processor):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for images, targets in pbar:
        images = images.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        
        # Prepare inputs for the model
        pixel_values = images
        task_prompt = "<s_rvlcdip>"
        decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
        decoder_input_ids = decoder_input_ids.repeat(images.size(0), 1)  # Repeat for batch size

        outputs = model(pixel_values=pixel_values, decoder_input_ids=decoder_input_ids.to(device), labels=targets)
        loss = outputs.loss
        logits = outputs.logits
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(logits.argmax(dim=-1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data config
data_path = 'dj/data/'

# Training config
img_size = 480  # Default size used in Donut
LR = 1e-4
EPOCHS = 5
BATCH_SIZE = 8  # Increased batch size to 8
num_workers = 4

# Augmentation for training transform
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Augmentation for testing transform
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Dataset definition
trn_dataset = ImageDataset(
    "dj/data/train.csv",
    "dj/data/train/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "dj/data/sample_submission.csv",
    "dj/data/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

# DataLoader definition
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

# Training loop
for epoch in range(EPOCHS):
    torch.cuda.empty_cache()
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device, processor=processor)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
        log += f"{k}: {v:.4f}\n"
    print(log)

# Evaluation
model.eval()
preds_list = []

for images, _ in tqdm(tst_loader):
    images = images.to(device)

    with torch.no_grad():
        outputs = model(pixel_values=images)
    preds_list.extend(outputs.logits.argmax(dim=-1).detach().cpu().numpy())

# Save predictions
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

sample_submission_df = pd.read_csv("dj/data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

pred_df.to_csv("pred_3.csv", index=False)
pred_df.head()


/opt/conda/envs/cv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/cv/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via

1570 3140


  0%|          | 0/197 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Loss: 3.3278: 100%|██████████| 197/197 [01:22<00:00,  2.38it/s]


...

train_loss: 0.1738
train_acc: 0.9580
train_f1: 0.9048
epoch: 4.0000



  0%|          | 0/393 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/393 [00:00<?, ?it/s]


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [5]:
import os
import random
from PIL import Image
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Set random seed for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# Load Donut processor and model
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = np.array(image)
            image = self.transform(image=image)['image']
            # Convert image back to [0, 1] range
            image = (image - image.min()) / (image.max() - image.min())

        return image, torch.tensor(target, dtype=torch.long)

# Train one epoch
def train_one_epoch(loader, model, optimizer, loss_fn, device, processor):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for images, targets in pbar:
        images = images.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        
        # Prepare inputs for the model
        task_prompt = "<s_rvlcdip>"
        decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
        decoder_input_ids = decoder_input_ids.repeat(images.size(0), 1).to(device)  # Repeat for batch size

        outputs = model(pixel_values=images, decoder_input_ids=decoder_input_ids, labels=targets)
        loss = outputs.loss
        logits = outputs.logits
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(logits.argmax(dim=-1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

# Validation function
def validate(loader, model, loss_fn, device, processor):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        pbar = tqdm(loader, leave=False)
        for images, targets in pbar:
            images = images.to(device)
            targets = targets.to(device)

            task_prompt = "<s_rvlcdip>"
            decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
            decoder_input_ids = decoder_input_ids.repeat(images.size(0), 1).to(device)  # Repeat for batch size

            outputs = model(pixel_values=images, decoder_input_ids=decoder_input_ids, labels=targets)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            preds_list.extend(logits.argmax(dim=-1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }

    return ret

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data config
data_path = 'dj/data/'

# Training config
img_size = 480  # Default size used in Donut
LR = 1e-4
EPOCHS = 50  # Increase the number of epochs to allow early stopping to work effectively
BATCH_SIZE = 8  # Increased batch size to 8
num_workers = 4
n_splits = 5  # Number of folds for stratified k-fold

# Early stopping config
early_stopping_patience = 5  # Number of epochs to wait before stopping if no improvement

# Augmentation for training transform
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Augmentation for testing transform
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Dataset definition
full_dataset = ImageDataset(
    "dj/data/train.csv",
    "dj/data/train/",
    transform=trn_transform
)
print(len(full_dataset))

# Stratified K-Fold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
targets = [label for _, label in full_dataset]

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(targets)), targets)):
    print(f"Fold {fold + 1}/{n_splits}")
    
    train_subsampler = Subset(full_dataset, train_idx)
    val_subsampler = Subset(full_dataset, val_idx)
    
    train_loader = DataLoader(
        train_subsampler,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )
    val_loader = DataLoader(
        val_subsampler,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    # Initialize model, optimizer, and loss function
    model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")
    model.to(device)
    optimizer = Adam(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss()
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Training loop with early stopping
    for epoch in range(EPOCHS):
        torch.cuda.empty_cache()
        train_ret = train_one_epoch(train_loader, model, optimizer, loss_fn, device, processor)
        val_ret = validate(val_loader, model, loss_fn, device, processor)
        
        log = f"Epoch {epoch + 1}/{EPOCHS} - " \
              f"Train Loss: {train_ret['train_loss']:.4f}, " \
              f"Train Acc: {train_ret['train_acc']:.4f}, " \
              f"Train F1: {train_ret['train_f1']:.4f}, " \
              f"Val Loss: {val_ret['val_loss']:.4f}, " \
              f"Val Acc: {val_ret['val_acc']:.4f}, " \
              f"Val F1: {val_ret['val_f1']:.4f}"
        print(log)
        
        # Check early stopping condition
        if val_ret['val_loss'] < best_val_loss:
            best_val_loss = val_ret['val_loss']
            patience_counter = 0
            torch.save(model.state_dict(), f"best_model_fold_{fold + 1}.pth")
        else:
            patience_counter += 1
        
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered")
            break
tst_dataset = ImageDataset(
    "dj/data/sample_submission.csv",
    "dj/data/test/",
    transform=tst_transform
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

# Load the best model and perform evaluation on the test set
best_model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip")
best_model.to(device)

# Load the best saved model for each fold
for fold in range(1, n_splits + 1):
    print(f"Evaluating Fold {fold}...")
    best_model.load_state_dict(torch.load(f"best_model_fold_{fold}.pth"))

    best_model.eval()
    preds_list = []

    for images, _ in tqdm(tst_loader):
        images = images.to(device)

        with torch.no_grad():
            # Prepare inputs for the model
            task_prompt = "<s_rvlcdip>"
            decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
            decoder_input_ids = decoder_input_ids.repeat(images.size(0), 1).to(device)  # Repeat for batch size

            outputs = best_model(pixel_values=images, decoder_input_ids=decoder_input_ids)
        preds_list.extend(outputs.logits.argmax(dim=-1).detach().cpu().numpy())

    # Save predictions
    pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = preds_list

    sample_submission_df = pd.read_csv("dj/data/sample_submission.csv")
    assert (sample_submission_df['ID'] == pred_df['ID']).all()

    pred_df.to_csv(f"pred_fold_{fold}.csv", index=False)
    print(pred_df.head())

print("Evaluation complete.")

/opt/conda/envs/cv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/cv/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via

1570
Fold 1/5


/opt/conda/envs/cv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/cv/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Loss: 1.9802: 100%|██████████| 157/157 [01:06<00:00,  2.32it/s]

...

                     ID target
0  0008fdb22ddce0ce.jpg    [2]
1  00091bffdffd83de.jpg   [11]
2  00396fbc1f6cc21d.jpg   [11]
3  00471f8038d9c4b6.jpg    [2]
4  00901f504008d884.jpg    [2]
Evaluating Fold 5...


  0%|          | 0/393 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|██████████| 393/393 [01:33<00:00,  4.21it/s]

                     ID target
0  0008fdb22ddce0ce.jpg    [2]
1  00091bffdffd83de.jpg   [10]
2  00396fbc1f6cc21d.jpg    [5]
3  00471f8038d9c4b6.jpg    [2]
4  00901f504008d884.jpg    [2]
Evaluation complete.


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import mode

# Load predictions from each fold
pred_df_list = [pd.read_csv(f"pred_fold_{i}.csv") for i in range(1, 6)]

# Check that all dataframes have the same IDs
for i in range(1, 5):
    assert (pred_df_list[0]['ID'] == pred_df_list[i]['ID']).all(), f"ID mismatch between fold 1 and fold {i+1}"

# Remove brackets from 'target' and convert to numeric values
for i, df in enumerate(pred_df_list):
    df['target'] = df['target'].apply(lambda x: int(x.strip('[]')))
    print(f"Fold {i+1} predictions:")
    print(df.head())

# Extract the 'target' values as a 2D list
preds = np.array([df['target'].values for df in pred_df_list])

# Print predictions to debug
print("Combined predictions from all folds:")
print(preds)

# Perform majority voting
voted_preds, _ = mode(preds, axis=0)

# Print voted predictions to debug
print("Voted predictions:")
print(voted_preds[0])

# Create final prediction dataframe
final_pred_df = pd.DataFrame({
    'ID': pred_df_list[0]['ID'],
    'target': voted_preds[0]
})

# Save the final predictions
final_pred_df.to_csv("dj/final_predictions.csv", index=False)
print(final_pred_df.head())


Fold 1 predictions:
                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      10
2  00396fbc1f6cc21d.jpg       5
3  00471f8038d9c4b6.jpg      10
4  00901f504008d884.jpg       2
Fold 2 predictions:
                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg       6
2  00396fbc1f6cc21d.jpg       8
3  00471f8038d9c4b6.jpg       0
4  00901f504008d884.jpg       2
Fold 3 predictions:
                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg       6
2  00396fbc1f6cc21d.jpg       5
3  00471f8038d9c4b6.jpg       2
4  00901f504008d884.jpg       2
Fold 4 predictions:
                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      11
2  00396fbc1f6cc21d.jpg      11
3  00471f8038d9c4b6.jpg       2
4  00901f504008d884.jpg       2
Fold 5 predictions:
                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      10
2  00396fbc1f6cc21d.jpg       5
3  0

In [2]:
import pandas as pd

pred_df = pd.read_csv("pred_fold_1.csv")

# Ensure the 'target' column is converted to the required format
pred_df['target'] = pred_df['target'].apply(lambda x: f'[{x}]')

# Save the modified predictions
pred_df.to_csv("dj/final_submission_fold_1.csv", index=False)

print(pred_df.head())

                     ID  target
0  0008fdb22ddce0ce.jpg   [[2]]
1  00091bffdffd83de.jpg  [[10]]
2  00396fbc1f6cc21d.jpg   [[5]]
3  00471f8038d9c4b6.jpg  [[10]]
4  00901f504008d884.jpg   [[2]]


In [3]:
import pandas as pd

# Load predictions from fold 1
pred_df = pd.read_csv("pred_fold_1.csv")

# Ensure the 'target' column is converted to the required format by removing brackets
pred_df['target'] = pred_df['target'].apply(lambda x: x.strip('[]'))

# Save the modified predictions
pred_df.to_csv("dj/final_submission_fold_1.csv", index=False)

print(pred_df.head())


                     ID target
0  0008fdb22ddce0ce.jpg      2
1  00091bffdffd83de.jpg     10
2  00396fbc1f6cc21d.jpg      5
3  00471f8038d9c4b6.jpg     10
4  00901f504008d884.jpg      2
